<a href="https://colab.research.google.com/github/ixomaxip/nasoc/blob/master/roBERTa-hate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Tue Aug 10 16:13:12 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%%capture

!pip install nltk emoji
!pip install tweet-preprocessor

!pip install transformers
!pip install datasets

!wget https://hasocfire.github.io/hasoc/2021/files/en_Hasoc2021_train.zip
!unzip -P hasoc2021_en en_Hasoc2021_train.zip
!ls


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# from google.colab import files
# uploaded = files.upload()

# !rm -rf /root/.ssh
# !mkdir /root/.ssh
# # !tar -xvzf ssh.tar.gz -C /root/.ssh && rm -rf ssh.tar.gz && chmod 700 /root/.ssh
# !tar -xvzf ssh.tar.gz -C /root/.ssh && rm -rf ssh.tar.gz
# !ssh-keyscan -t rsa github.com >> /root/.ssh/known_hosts

# !git config --global user.email 'ixomaxip@gmail.com'
# !git config --global user.user 'ixomaxip'
# !ssh -T git@github.com

# !git clone git@github.com:ixomaxip/nasoc.git

In [4]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report

# Datasets

In [ ]:
FULL_PATH = '/content/nasoc/data/en_Hasoc2021_train.csv'
TRAIN_PATH = '/content/nasoc/data/train.csv'
TEST_PATH = '/content/nasoc/data/validate.csv'

full_df = pd.read_csv(FULL_PATH)
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

print(f'''shapes:
  full: {full_df.shape}
  train: {train_df.shape}
  test: {test_df.shape}
''')
print(f'''classes:
  task_1: {full_df.task_1.unique()}
  task_2: {full_df.task_2.unique()}
''')
train_df.columns

shapes:
  full: (3843, 5)
  train: (3074, 7)
  test: (769, 7)

classes:
  task_1: ['HOF' 'NOT']
  task_2: ['PRFN' 'OFFN' 'NONE' 'HATE']



Index(['Unnamed: 0', '_id', 'text', 'task_1', 'task_2', 'inputs',
       'task_2_cat'],
      dtype='object')

In [ ]:
!wget https://setup.johnsnowlabs.com/nlu/colab.sh -O - | bash 
import nlu
# import nlu-gpu

In [ ]:
def prepare_df(path, task):
  df = pd.read_csv(path)
  df.rename(columns={f'task_{task}': 'y'}, inplace=True)
  df = df[['text', 'y']]
  return df

train_t1 = prepare_df(TRAIN_PATH, task=1)
test_t1 = prepare_df(TEST_PATH, task=1)
train_t1.y.unique()

NameError: ignored

In [ ]:
trainable_pipe = nlu.load('xx.embed_sentence.labse train.sentiment')
trainable_pipe['sentiment_dl'].setMaxEpochs(60)  
trainable_pipe['sentiment_dl'].setLr(0.005)
fitted_pipe = trainable_pipe.fit(train_t1)
preds = fitted_pipe.predict(train_t1, output_level='document')
preds.dropna(inplace=True)

print(classification_report(preds['y'], preds['trained_sentiment']))
MODEL_PATH = './drive/MyDrive/nasoc/task1/sentiment_dl_baseline_trained'
fitted_pipe.save(MODEL_PATH)
preds

Mounted at /content/drive


In [ ]:
preds = fitted_pipe.predict(test_t1, output_level='document')
preds.dropna(inplace=True)
print(classification_report(preds['y'], preds['trained_sentiment']))

In [ ]:

import torch
from transformers import AutoTokenizer, AutoModel

DEVICE = torch.device('cuda:0')
tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")
# model = AutoModel.from_pretrained("cointegrated/LaBSE-en-ru")
# model.eval().to(DEVICE)

In [ ]:
def text_encoder(texts, model, tokenizer, max_len=128):
    encoded_input = tokenizer(texts, padding=True, truncation=True, max_length=max_len, return_tensors='pt').to(device=DEVICE)
    with torch.no_grad():
        model_output = model(**encoded_input)
    embeddings = model_output.pooler_output
    embeddings = torch.nn.functional.normalize(embeddings)

    return embeddings

In [ ]:
ROOT_DIR = '/content/drive/MyDrive/nasoc/'
TRAIN_EMB = os.path.join(ROOT_DIR, 'train_emb.feather')
TEST_EMB = os.path.join(ROOT_DIR, 'test_emb.feather')

## Get labse embeddings

In [ ]:
# train_df['labse_emb'] = train_df.apply(lambda row: text_encoder(row['text'], model, tokenizer, 64)[0].detach().cpu().numpy(), axis=1)
# train_df.to_feather(TRAIN_EMB)

# test_df['labse_emb'] = test_df.apply(lambda row: text_encoder(row['text'], model, tokenizer, 64)[0].detach().cpu().numpy(), axis=1)
# test_df.to_feather(TEST_EMB)
train_df = pd.read_feather(TRAIN_EMB)
test_df = pd.read_feather(TEST_EMB)

In [ ]:
test_df.head(2)

,Unnamed: 0,_id,text,task_1,task_2,inputs,task_2_cat,labse_emb
0,4745,60c5d6bf5659ea5e55def8b8,4lakh cases in a day ðŸ˜” @PMOIndia @drharshva...,HOF,OFFN,0,2,"[-0.02545384, 0.03337645, 0.020745805, -0.0581..."
1,4795,60c5d6bf5659ea5e55def871,@SiennaSummerEIC @Chantel_Etoile You do unders...,HOF,PRFN,0,3,"[0.027386742, -0.0026722655, 0.0048503056, -0...."


## XGBOOST

In [ ]:
import xgboost as xgb

In [ ]:
x_train = np.stack(train_df.labse_emb.to_numpy())
y_train = train_df.inputs.values
print(x_train.shape)

x_test = np.stack(test_df.labse_emb.to_numpy())
y_test = test_df.inputs.values

(3074, 768)


In [ ]:
bst = xgb.XGBClassifier(
    objective='reg:logistic'
)

bst.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='reg:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
y_pred = bst.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.89      0.84       529
           1       0.66      0.47      0.55       240

    accuracy                           0.76       769
   macro avg       0.73      0.68      0.69       769
weighted avg       0.75      0.76      0.75       769



In [ ]:
row = train_df.iloc[0]
text = row.text
print(text)
emb = text_encoder(row['text'], model, tokenizer, 64)[0].detach().cpu().numpy()

@Turbo_Fucker Too bad I'm a dummy not an idiots


# Custom BERTweet

In [5]:

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import os
import datetime
import re

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

#constants and parameters
seed = 42

path_gdrive_saved = '/content/drive/My Drive/nasoc/bert_tuned.weights'

batch_size = 8

DOWNLOAD_DATASET = False

raw_dataset = pd.read_csv('en_Hasoc2021_train.csv')

raw_dataset['inputs'] = pd.Categorical(raw_dataset['task_1'])
raw_dataset['inputs'] = raw_dataset.inputs.cat.codes

raw_dataset['task_2_cat'] = pd.Categorical(raw_dataset['task_2'])
raw_dataset['task_2_cat'] = raw_dataset.task_2_cat.cat.codes

train_df, test_df = train_test_split(raw_dataset, test_size=0.2, random_state=seed)

sr_train = train_df.groupby('task_1').count()['text']
sr_test  = test_df.groupby('task_1').count()['text']

tr_pr = sr_train['NOT']/sr_train['HOF']
ts_pr = sr_test['NOT']/sr_test['HOF']

#copied from https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

# copied from https://stackoverflow.com/questions/64719706/cleaning-twitter-data-pandas-python
def cleaner(tweet):
    tweet = re.sub("@[A-Za-z0-9]+","",tweet) #Remove @ sign
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
    #tweet = ''.join(c for c in tweet if c not in emoji.UNICODE_EMOJI) #Remove Emojis
    tweet = tweet.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
    tweet = " ".join(tweet.split())
    tweet = remove_emojis(tweet)
    #tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet) \
    #     if w.lower() in words or not w.isalpha())
    return tweet


train_df, test_df = train_test_split(raw_dataset, test_size=0.2, random_state=seed)
#text = train_df.pop('text')#.map(lambda line: cleaner(line))
#label = train_df.pop('inputs')

#text_val = test_df.pop('text')#.map(lambda line: cleaner(line))
#label_val = test_df.pop('inputs')

#train_ds = tf.data.Dataset.from_tensor_slices((text.values, label.values))
#val_ds = tf.data.Dataset.from_tensor_slices((text_val.values, label_val.values))

#train_ds = train_ds.batch(batch_size=batch_size, drop_remainder=True)
#val_ds = val_ds.batch(batch_size=batch_size, drop_remainder=True)

In [6]:
import time, datetime
import random
import numpy as np

import torch
from torch import nn
from torch.nn import functional as F, Linear
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from transformers import AutoTokenizer, AutoModel, AdamW
from transformers import get_linear_schedule_with_warmup

## Dataloader

In [12]:
class HasocDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.encodings = tokenizer(texts, padding=True, truncation=True, max_length=64, return_tensors='pt')
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

MODEL_NAME = "cardiffnlp/twitter-roberta-base-hate"
#vinai/bertweet-base

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
train_dataset = HasocDataset(train_df['text'].to_list(), train_df['inputs'].to_list(), tokenizer)
test_dataset = HasocDataset(test_df['text'].to_list(), test_df['inputs'].to_list(), tokenizer)

BATCH_SIZE = 16
train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset),
            batch_size = BATCH_SIZE
)
test_dataloader = DataLoader(
            test_dataset,
            sampler = RandomSampler(test_dataset),
            batch_size = BATCH_SIZE
)
len(train_dataloader)

193

In [13]:
it = iter(test_dataloader)
first = next(it)

print(first)

{'input_ids': tensor([[    0,  1039,    90,  ...,     1,     1,     1],
        [    0,  1039, 17488,  ...,   787, 46474,     2],
        [    0, 35702,     5,  ...,   849, 30597,     2],
        ...,
        [    0, 10431, 11015,  ...,     2,     1,     1],
        [    0,  1039,   282,  ...,     1,     1,     1],
        [    0,  9690, 19124,  ...,   100, 37074,     2]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1])}


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


## Customization class & auxiliary functions

In [14]:
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# DEVICE = torch.device('cpu')

# Customization
class CustomBERTweet(nn.Module):
    def __init__(self):
        super(CustomBERTweet, self).__init__()
        self.bertweet = AutoModel.from_pretrained(MODEL_NAME)
        self.dropout = nn.Dropout(0.1)
        self.dense1 = nn.Linear(768, 2)
        # self.dense2 = nn.Linear(256, 2)
    
    def forward(self, feed_dict):

        bertweet_output = self.bertweet(**feed_dict)
        drop_output = self.dropout(bertweet_output.pooler_output)
        linear1_output = self.dense1(drop_output)
        # linear2_output = self.dense2(linear1_output)

        return linear1_output

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

def forward(batch, model):
    feed_dict = {
        'input_ids': batch['input_ids'].to(DEVICE),
        #'token_type_ids': batch['token_type_ids'].to(DEVICE),
        'attention_mask': batch['attention_mask'].to(DEVICE),
    }
    # feed_dict = {
    #     'input_ids': batch['input_ids'].to(DEVICE),
    #     'token_type_ids': batch['token_type_ids'].to(DEVICE),
    #     'attention_mask': batch['attention_mask'].to(DEVICE),
    # }    
    outputs = model(feed_dict)
    outputs = F.log_softmax(outputs, dim=1)

    return outputs

def validate(model, dataloader):
    predictions, targets = [], []
    for batch in dataloader:
        target = batch['labels']
        with torch.no_grad():
            logits = forward(batch, model)
        logits = logits.detach().cpu().numpy()
        label = target.to('cpu').numpy()        
        predictions.append(logits)
        targets.append(label)
    flat_predictions = [item for sublist in predictions for item in sublist]
    flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
    flat_targets = [item for sublist in targets for item in sublist]
    return classification_report(flat_targets, flat_predictions)


## Create model

In [15]:
model = CustomBERTweet()

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-hate were not used when initializing RobertaModel: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-hate and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inf

## Check model

In [16]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())
print(f'The BERT model has {len(params):} different named parameters.\n')
print('==== Embedding Layer ====\n')
for p in params[0:5]:
    print(f'{p[0]:<55} {str(tuple(p[1].size())):>12}')
print('\n==== First Transformer ====\n')
for p in params[5:21]:
    print(f'{p[0]:<55} {str(tuple(p[1].size())):>12}')
print('\n==== Output Layer ====\n')
for p in params[-12:]:
    print(f'{p[0]:<55} {str(tuple(p[1].size())):>12}')

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bertweet.embeddings.word_embeddings.weight              (50265, 768)
bertweet.embeddings.position_embeddings.weight            (514, 768)
bertweet.embeddings.token_type_embeddings.weight            (1, 768)
bertweet.embeddings.LayerNorm.weight                          (768,)
bertweet.embeddings.LayerNorm.bias                            (768,)

==== First Transformer ====

bertweet.encoder.layer.0.attention.self.query.weight      (768, 768)
bertweet.encoder.layer.0.attention.self.query.bias            (768,)
bertweet.encoder.layer.0.attention.self.key.weight        (768, 768)
bertweet.encoder.layer.0.attention.self.key.bias              (768,)
bertweet.encoder.layer.0.attention.self.value.weight      (768, 768)
bertweet.encoder.layer.0.attention.self.value.bias            (768,)
bertweet.encoder.layer.0.attention.output.dense.weight    (768, 768)
bertweet.encoder.layer.0.attention.output.dense.bias          (

In [17]:
model.to(torch.device('cuda'))
print(validate(model, test_dataloader))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


              precision    recall  f1-score   support

           0       0.68      0.05      0.10       529
           1       0.31      0.95      0.47       240

    accuracy                           0.33       769
   macro avg       0.50      0.50      0.28       769
weighted avg       0.57      0.33      0.21       769



## Train

In [18]:
EPOCHS = 4
total_steps = len(train_dataloader) * EPOCHS
optimizer = AdamW(model.parameters(), lr = 2e-5, eps = 1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 20,
                                            num_training_steps = total_steps)
loss_func = nn.CrossEntropyLoss()

model.to(DEVICE)
start = time.time()
for epoch in range(0, EPOCHS):
    total_train_loss = 0
    model.train()
    print(f'======== Epoch {epoch+1} / {EPOCHS} ========')
    for step, batch in enumerate(train_dataloader):
        if step % 50 == 0 and not step == 0:
            elapsed = format_time(time.time() - start)
            print(f'\tBatch {step:>5,}  of  {len(train_dataloader):>5,}.    Elapsed: {elapsed}.')
            # print(f'\tloss: {loss.item()}')
        optimizer.zero_grad()
        outputs = forward(batch, model)
        target = batch['labels'].to(DEVICE)
        model.zero_grad()
        loss = loss_func(outputs, target)
        total_train_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
    
    avg_train_loss = total_train_loss / len(train_dataloader)
    training_time = format_time(time.time() - start)
    print(f'\tAverage training loss: {avg_train_loss:.2f}')
    print(f'\tTraining epoch took: {training_time}')
    model.eval()
    report = validate(model, test_dataloader)
    print(f'======== Validation report {epoch+1} / {EPOCHS} ========')
    print(report)


======== Epoch 1 / 4 ========


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


	Batch    50  of    193.    Elapsed: 0:00:06.
	Batch   100  of    193.    Elapsed: 0:00:13.
	Batch   150  of    193.    Elapsed: 0:00:19.
	Average training loss: 0.51
	Training epoch took: 0:00:25
======== Validation report 1 / 4 ========
              precision    recall  f1-score   support

           0       0.82      0.95      0.88       529
           1       0.84      0.54      0.66       240

    accuracy                           0.82       769
   macro avg       0.83      0.75      0.77       769
weighted avg       0.83      0.82      0.81       769

======== Epoch 2 / 4 ========
	Batch    50  of    193.    Elapsed: 0:00:33.
	Batch   100  of    193.    Elapsed: 0:00:39.
	Batch   150  of    193.    Elapsed: 0:00:46.
	Average training loss: 0.35
	Training epoch took: 0:00:51
======== Validation report 2 / 4 ========
              precision    recall  f1-score   support

           0       0.85      0.93      0.89       529
           1       0.80      0.65      0.72       240

 

In [ ]:
# custom_model = CustomLaBSE()
# custom_model.eval()
# encoded = tokenizer(train_texts[:10], padding=True, truncation=True, max_length=64, return_tensors='pt')
# with torch.no_grad():
  # outputs = custom_model.forward(encoded)
  # labse_output = custom_model.labse(**encoded)
# dense1 = nn.Linear(768, 256)
# dense2 = nn.Linear(256, 2)
# linear1_output = dense1(labse_output.pooler_output)
# linear2_output = dense2(linear1_output)
# crit = nn.CrossEntropyLoss()
# sm = F.log_softmax(outputs, dim=1)
# target = torch.tensor(train_labels[:10])
# crit(sm, target)

In [19]:
model.eval()
report = validate(model, test_dataloader)
print(report)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


              precision    recall  f1-score   support

           0       0.86      0.91      0.88       529
           1       0.77      0.68      0.72       240

    accuracy                           0.84       769
   macro avg       0.82      0.80      0.80       769
weighted avg       0.83      0.84      0.83       769



#Garbage

In [ ]:
from transformers import Trainer, TrainingArguments
OUTPUT_DIR = os.path.join(ROOT_DIR, 'labse_ft_results')

training_args = TrainingArguments(
    output_dir = OUTPUT_DIR,          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=0,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=os.path.join(OUTPUT_DIR, 'logs'),            # directory for storing logs
    logging_steps=10,
)

trainer = Trainer(
    model=custom_model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset             # evaluation dataset
)
trainer.train()

***** Running training *****
  Num examples = 3074
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 579
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


TypeError: ignored

In [ ]:
max_len = 64
row = train_df.iloc[0]
texts = row.text
encoded_input = tokenizer([texts, texts], padding=True, truncation=True, max_length=max_len, return_tensors='pt').to(device=DEVICE)
with torch.no_grad():
    model_output = model(**encoded_input)
embeddings = model_output.pooler_output
embeddings = torch.nn.functional.normalize(embeddings)

In [ ]:
model_output.pooler_output.shape

torch.Size([2, 768])

In [ ]:
train_embedings = text_encoder(train_texts, model, tokenizer, 64)

In [ ]:
sentences = ["Hello World"]
encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=max_len, return_tensors='pt').to(DEVICE)
with torch.no_grad():
    model_output = model(**encoded_input)
embeddings = model_output.pooler_output
embeddings = torch.nn.functional.normalize(embeddings)
# print(embeddings[0])

In [ ]:
model_output.pooler_output.shape

torch.Size([1, 768])

In [ ]:
model.parameters()

<generator object Module.parameters at 0x7f24480df4d0>

In [ ]:
len(train_labels)

3074

In [ ]:


train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)